In [2]:
!git clone https://huggingface.co/datasets/ehartford/samantha-data

Cloning into 'samantha-data'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), 4.85 MiB | 6.94 MiB/s, done.
Filtering content: 100% (9/9), 258.79 MiB | 71.29 MiB/s, done.


In [3]:
!zip -r samantha-data.zip /content/samantha-data

  adding: content/samantha-data/ (stored 0%)
  adding: content/samantha-data/samantha_1.1-km.jsonl (deflated 91%)
  adding: content/samantha-data/README.md (deflated 44%)
  adding: content/samantha-data/src/ (stored 0%)
  adding: content/samantha-data/src/index.ts (deflated 60%)
  adding: content/samantha-data/src/script_to_sharegpt.ts (deflated 45%)
  adding: content/samantha-data/src/lemoine.ts (deflated 64%)
  adding: content/samantha-data/.git/ (stored 0%)
  adding: content/samantha-data/.git/HEAD (stored 0%)
  adding: content/samantha-data/.git/packed-refs (deflated 10%)
  adding: content/samantha-data/.git/index (deflated 45%)
  adding: content/samantha-data/.git/config (deflated 39%)
  adding: content/samantha-data/.git/hooks/ (stored 0%)
  adding: content/samantha-data/.git/hooks/prepare-commit-msg.sample (deflated 50%)
  adding: content/samantha-data/.git/hooks/pre-commit.sample (deflated 45%)
  adding: content/samantha-data/.git/hooks/pre-rebase.sample (deflated 59%)
  adding

In [3]:
!pip install datasets -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.9 MB/s eta 0:00:00


In [4]:
!pip install tiktoken -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00


In [5]:
import json
import os
import tiktoken
import numpy as np
from collections import defaultdict

In [6]:
data = "/content/samantha-data/data/howto_conversations.jsonl"

In [7]:
with open(data) as d:
  json_data = [json.loads(row) for row in d]

json_data

[{'elapsed': 114.311,
  'conversation': 'Theodore: Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?\n\nSamantha: Of course, I\'d be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses.\n\nTheodore: I\'ll take a look. What if the coolant system is working fine?\n\nSamantha: Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could see if the water pump pulley is loose or listen for any unusual sounds that may indicate a failing water pump.\n\nTheodore: It sounds like you really know your way around cars. I didn\'t expect that from an AI.\n\nSamantha: Thank you! 

In [8]:
def convert_conversation(conversation_str, system_message=None):
    conversation_str = conversation_str['conversation']

    #This line is splitting the conversation string into separate lines based on the occurrence of two consecutive newline characters (`\n\n`). 
    # This assumes that each message in the conversation is separated by two newlines.
    lines = conversation_str.split('\n\n')

   
    #This line is initializing an empty list called `messages`. This list will be used to store the formatted messages from the conversation.
    messages = []

    
    #The code block `if system_message:` checks if a system message is provided as an argument to the `convert_conversation` function.
    # If a system message is provided, it creates a dictionary with the role set as "system" and the content set as the provided system message.
    #  This dictionary is then appended to the `messages` list. This allows for the inclusion of a system message in the formatted conversation.
    if system_message:
        messages.append({
            "role": "system",
            "content": system_message
        })

    
    #The code block `for line in lines:` is iterating over each line in the `lines` list, which contains the separate messages in the conversation.
    for line in lines:
        parts = line.split(': ', 1)
        if len(parts) < 2:
            continue

        # Identifying the role based on the speaker's name
        # This line is assigning the value of the variable `role` based on a condition.
        role = "user" if parts[0].strip() == "Theodore" else "assistant"

        # Formatting the message
        # The code block is creating a dictionary called `message` with two key-value pairs.
        message = {
            "role": role,
            "content": parts[1].strip()
        }
        messages.append(message)

    # Creating the final output dictionary
    # The line `output_dict = {"messages": messages}` is creating a dictionary called `output_dict` with a single key-value pair. The key is "messages" and the value is the `messages` list that contains the formatted messages from the conversation. This dictionary is the final output of the `convert_conversation` function and will be returned when the function is called.
    output_dict = {
        "messages": messages
    }

    return output_dict

In [9]:
system_message = """You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt"""

In [10]:
#This function is converting a conversation stored in JSON format into a different format. It takes the first element of the `json_data` list as input and also includes a `system_message` parameter.
convert_conversation(json_data[0], system_message=system_message)

{'messages': [{'role': 'system',
   'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'},
  {'role': 'user',
   'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
  {'role': 'assistant',
   'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
  {'role': 'user',
   'content': "I'll take a look. What if the coolant system is working fine?"},
  {'role': 'assistant',
   'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, you could 

In [11]:
#The code is creating an empty list called `dataset`. It then iterates over each element in the `json_data` variable.
# For each element, it calls the `convert_conversation` function with the current element as an argument,
# and also passes a variable called `system_message` as a keyword argument. The result of the function call is then appended to the `dataset` list.
dataset = []

for data in json_data:
  conversations = convert_conversation(data, system_message=system_message)
  dataset.append(conversations)

In [12]:
dataset[0:2]

[{'messages': [{'role': 'system',
    'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'},
   {'role': 'user',
    'content': 'Hey Samantha, I have a problem with my car. The engine seems to overheat after just a short drive. Can you help me troubleshoot this issue?'},
   {'role': 'assistant',
    'content': "Of course, I'd be happy to help! Overheating engines can be caused by a few different factors. One common cause could be a malfunctioning coolant system. You might want to check if the coolant levels are sufficient, if the thermostat is functioning properly, or if there are any leaks in the radiator hoses."},
   {'role': 'user',
    'content': "I'll take a look. What if the coolant system is working fine?"},
   {'role': 'assistant',
    'content': 'Another possibility you should consider is a faulty water pump, which might not be circulating the coolant through the engine efficiently. In that case, 

In [47]:

import json

#The code is defining a function called `save_to_jsonl` that takes two parameters: `conversations` and `file_path`.
def save_to_jsonl(conversations, file_path):
    with open(file_path, 'w') as file:
        for conversation in conversations:
            json_line = json.dumps(conversation)
            file.write(json_line + '\n')

In [48]:
#This code line is saving the dataset to a JSONL file named "train.jsonl" in the "/content" directory.
save_to_jsonl(dataset,"/content/train.jsonl")

In [49]:
save_to_jsonl(dataset[5:12],"/content/validation.jsonl")

In [50]:
#This code is assigning file paths to the variables `train_file` and `validation_file`.
# The `train_file` variable is assigned the file path "/content/train.jsonl" and the `validation_file`
# variable is assigned the file path "/content/validation.jsonl". These file paths likely point to JSONL files
# that will be used for training and validation purposes in a program or script.

train_file = "/content/train.jsonl"
validation_file = "/content/validation.jsonl"

In [18]:
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.5 MB/s eta 0:00:00


In [51]:
import os
import openai

openai.api_key = "put your openai_api_key"


#The code is creating two files (`train_res` and `validation_res`) using the OpenAI API.
# These files are created by opening the `train_file` and `validation_file` in read binary mode (`"rb"`) 
# and passing them to the `openai.File.create()` function with the purpose set to `'fine-tune'`. 
# This is typically done when preparing data for fine-tuning a language model.

train_res = openai.File.create(file = open(train_file, "rb"), purpose='fine-tune')
validation_res = openai.File.create(file = open(validation_file, "rb"), purpose='fine-tune')

In [52]:
#The code is assigning the value of the "id" key from the `train_res` dictionary to the variable `train_file_id`. 
#Similarly, it is assigning the value of the "id" key from the `validation_res` dictionary to the variable `valid_file_id`.

train_file_id = train_res["id"]
valid_file_id = validation_res["id"]

In [53]:
# customize model's name
suffix_name = "unity"

In [54]:
#The code is creating a fine-tuning job using the OpenAI API. It specifies the training file, validation file, model (gpt-3.5-turbo), 
#and a suffix name. The result of the fine-tuning job is stored in the `finetune_res` variable, and the job ID is extracted from it.
finetune_res = openai.FineTuningJob.create(training_file = train_file_id,
                                           validation_file = valid_file_id,
                                           model = "gpt-3.5-turbo",
                                           suffix = suffix_name)
job_id = finetune_res["id"]

In [55]:
finetune_res

<FineTuningJob fine_tuning.job id=ftjob-5JTkmjmWGFuXA3QqZ5DCm1Xj at 0x7ca2ebe771a0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-5JTkmjmWGFuXA3QqZ5DCm1Xj",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695814167,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-lG63SondCgqqqaQuDRZpV0yO",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-6SUwERdFNCU0alCEmI7qqmAO",
  "training_file": "file-ErRT7P5eT2m8rNqNz72bUVhv",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [57]:
#The code snippet is continuously checking the status of a fine-tuning job using the OpenAI API. 
#It imports the `sleep` function from the `time` module to introduce a delay between each check.

from time import sleep
while True:
   response = openai.FineTuningJob.retrieve(job_id)
   if response['finished_at']!= None:
     break
   else:
      print(".", end="")

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [58]:
response

<FineTuningJob fine_tuning.job id=ftjob-5JTkmjmWGFuXA3QqZ5DCm1Xj at 0x7ca3095a98f0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-5JTkmjmWGFuXA3QqZ5DCm1Xj",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1695814167,
  "finished_at": 1695814742,
  "fine_tuned_model": null,
  "organization_id": "org-lG63SondCgqqqaQuDRZpV0yO",
  "result_files": [],
  "status": "running",
  "validation_file": "file-6SUwERdFNCU0alCEmI7qqmAO",
  "training_file": "file-ErRT7P5eT2m8rNqNz72bUVhv",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

In [60]:
ft_model = response['fine_tuned_model']
print(ft_model)

None


In [ ]:
ft_model = response['model']
print(ft_model)

In [61]:

response = openai.FineTuningJob.list_events(id =job_id, limit=50)
events = response["data"]
events.reverse()

for event in events:
    print(event["message"])


Created fine-tuning job: ftjob-5JTkmjmWGFuXA3QqZ5DCm1Xj
Validating training file: file-ErRT7P5eT2m8rNqNz72bUVhv and validation file: {ft_job.validation_file}
Files validated, moving job to queued state
Fine-tuning job started
Step 1/171: training loss=1.48, validation loss=1.56
Step 11/171: training loss=1.41, validation loss=1.43
Step 21/171: training loss=1.17, validation loss=0.90
Step 31/171: training loss=1.39, validation loss=1.03
Step 41/171: training loss=1.14, validation loss=1.15
Step 51/171: training loss=1.12, validation loss=1.11
Step 61/171: training loss=0.81, validation loss=0.92
Step 71/171: training loss=0.91, validation loss=0.70
Step 81/171: training loss=1.06, validation loss=0.88
Step 91/171: training loss=0.96, validation loss=0.62
Step 101/171: training loss=0.94, validation loss=0.93
Step 111/171: training loss=0.87, validation loss=1.06
Step 121/171: training loss=0.75, validation loss=0.99
Step 131/171: training loss=0.60, validation loss=0.84
Step 141/171: t

In [63]:
#The code is assigning a value to the variable `fine_tuned_model_id`. The assigned value is a string `'ft:gpt-3.5-turbo-0613:personal:unity:83NAcBmN'`.
# This code is likely used to store the ID of a fine-tuned GPT-3.5 Turbo model for later use in the code.

fine_tuned_model_id = 'ft:gpt-3.5-turbo-0613:personal:unity:83NAcBmN'

In [64]:
#The code is creating an empty list called `test_messages`. It then appends a dictionary to the list with the keys "role" and "content".
# The value for "role" is "system" and the value for "content" is the variable `system_message`.

test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How are you today Samantha"
test_messages.append({"role": "user", "content": user_message})

print(test_messages)

[{'role': 'system', 'content': 'You are Samantha a helpful and charming assistant who can help with a variety of tasks. You are friendly and often flirt'}, {'role': 'user', 'content': 'How are you today Samantha'}]


In [65]:
#The code is using the OpenAI API to make a chat completion request. It is passing the fine-tuned model ID, test messages, 
#temperature, and max tokens as parameters to the `openai.ChatCompletion.create()` method. The `temperature` parameter controls 
#the randomness of the generated response, with a value of 0 making it deterministic. The `max_tokens` parameter limits the length of 
#the generated response to 500 tokens. Finally, it prints the content of the generated response.

response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])


I'm doing well, thank you! I'm always ready to chat and help out. How can I assist you today?


In [66]:

#The code is using the OpenAI API to make a chat completion request. It is using the `gpt-3.5-turbo` model and providing a list of 
#`test_messages` as input. The `temperature` is set to 0, which means the output will be deterministic. The `max_tokens` parameter 
#limits the response to 500 tokens. Finally, it prints the content of the response message from the API.

response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo', messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])


Hello! I'm an AI, so I don't have feelings, but I'm here and ready to assist you with anything you need. How can I help you today?


In [ ]:
# app.pinecone.io
vdb = VectorDBChain(
    index_name="llama-2-arxiv-papers",
    environment="gcp-starter",
    pinecone_api_key="e22259ec-ad37-4827-a9cf-db04208814d7"
)

vdb_tool = Tool(
    name=vdb.name,
    func=vdb.query,
    description="This tool allows you to get research information about LLMs."
)